In [ ]:
# For Google Colaboratory

!pip install torchinfo
!unzip training.zip

In [ ]:

%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.image as mpimg
from torch import nn
from torch import Tensor
from torch.utils.data import TensorDataset, DataLoader
from torchvision import transforms
from torchinfo import summary
import torch
import numpy as np
import matplotlib.pyplot as plt
import os, sys
from PIL import Image
from utils import read_all_images, img_crop
from augmentation import augment_data, rot90, rot180, rot270, fliph, flipv, flipvh

from road_segmentation.utils import *

In [ ]:
import warning

device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cpu":
    warning.warn("CUDA not detected, training will be very slow")

device = torch.device(device)

# Dataset loading

In [ ]:
IMAGE_SRC_DIR = "data/training/images/"
IMAGE_GT_DIR = "data/training/groundtruth/"

IMAGES_SRC = np.asarray(read_all_images(IMAGE_SRC_DIR))
IMAGES_GT = np.asarray(read_all_images(IMAGE_GT_DIR))

IMAGES_SRC.shape

# Augment data

Testing multiple examples

In [ ]:
visualize_augmented_data(IMAGES_SRC, IMAGES_GT)

In [ ]:
augmented_src, augmented_gt = augment_data(IMAGES_SRC, IMAGES_GT)
#augmented_src, augmented_gt = IMAGES_SRC, IMAGES_GT
print(augmented_src.shape)
print(augmented_gt.shape)

## crop

In [ ]:
PATCH_SIZE = 100
cropped_src, cropped_gt = np.asarray([img_crop(img, PATCH_SIZE, PATCH_SIZE) for img in augmented_src]).reshape((-1, PATCH_SIZE, PATCH_SIZE, 3)), np.asarray([img_crop(img, PATCH_SIZE, PATCH_SIZE) for img in augmented_gt]).reshape((-1, PATCH_SIZE, PATCH_SIZE))
print(cropped_src.shape)
print(cropped_gt.shape)

# Normalization

In [ ]:
def get_mean_std(imgs):
  """
  Normalizes images with mean and standard deviation, by channel
  """
  mean = imgs.mean(axis=(0, 1, 2))
  std = imgs.std(axis=(0, 1, 2))

  return mean, std

means, stds = get_mean_std(cropped_src)

# Dataset loading

In [ ]:
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(means, stds)
])

ds_src, ds_gt = torch.stack([img_transform(i) for i in cropped_src]), torch.stack([transforms.ToTensor()(i) for i in cropped_gt]).squeeze()
print(ds_src.shape)
print(ds_gt.shape)

In [ ]:
dataset = TensorDataset(ds_src, ds_gt)

## + splitting

In [ ]:
def split_dataset(dataset):
  train_size = int(len(dataset) * 0.8)
  test_size = len(dataset) - train_size
  return torch.utils.data.random_split(dataset, [train_size, test_size])

train_ds, test_ds = split_dataset(dataset)

# Model definition

In [ ]:
from models.UNet import UNet

model=UNet(n_channels=3, n_classes=1)
summary(model, input_size=(6, 3, PATCH_SIZE, PATCH_SIZE))


# Training!

In [ ]:
from tqdm.notebook import tqdm
def train_epoch(model, device, train_loader, optimizer, epoch, criterion):
    model.train()

    loss_history = []
    acc_history = []
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model.forward(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        loss_history.append(loss.item())
        acc_history.append(((output > 0).eq(target > 0).sum() / target.numel()).item())

    return acc_history, loss_history

In [ ]:
def train(device, batch_size=32, epochs=60):
    pytorchDl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    criterion = nn.BCEWithLogitsLoss()

    for i in tqdm(range(epochs)):
        acc_history, loss_history = train_epoch(model, device, pytorchDl, optimizer, i, criterion)
        if i % 1 == 0:
          print("Epoch: ", i, " Accuracy: ", sum(acc_history)/len(acc_history), " Loss: ", sum(loss_history)/len(loss_history))
        if i%5==0:
          print("saving…")
          torch.save(model.state_dict(), "./checkpoint_{}.pth".format(i))
        if i%5 == 0:
          f1 = compute_f1(test_ds, model, device, threshold=0.25)
          print("F1: ", f1)

    return

In [ ]:
model.load_state_dict(torch.load("./UNet_145.pth"))

# Let's play!

In [ ]:
#train(device, 32, 150)

# Visualize results?

In [ ]:
visualize_result(device, model, train_ds)
visualize_result(device, model, test_ds)

In [ ]:
dl = DataLoader(train_ds, batch_size=1, shuffle=True)

N = 1000

acc_dict = {}

for batch_idx, (data, target) in enumerate(dl):
    data, target = data.to(device), target.to(device)
    output = model.forward(data)

    acc = ((output > 0).eq(target > 0).sum() / target.numel()).item()

    acc_dict[acc] = (data, target)

    if batch_idx >= N:
      break

k = 4
top_k = []

accs = sorted(list(acc_dict.keys()))

top_k = [
    (accs[i], acc_dict[accs[i]])
    for i in range(min(len(accs), k))
]

m = len(top_k)

from math import ceil
cols = 3*2 # 2 sets of 3 images
rows = ceil(m/2)

fig = plt.figure(figsize=(15, 15))

for i, (acc, (data, target)) in enumerate(top_k):
    idx = i*3

    data = data.squeeze()
    target = target.squeeze()

    inputs = data.to(device).unsqueeze(0)

    output = model.forward(inputs)

    data, target = data.cpu().numpy().transpose((1, 2, 0)), target.cpu().numpy()

    fig.add_subplot(rows, cols, idx+1)
    im = plt.imshow(data)
    plt.title("Original Image")

    fig.add_subplot(rows, cols, idx+2)
    im = plt.imshow(target)
    fig.colorbar(im, fraction=0.046, pad=0.04)
    plt.title("Ground Truth")

    fig.add_subplot(rows, cols, idx+3)
    im = plt.imshow(nn.Sigmoid()(output.cpu().detach()).numpy())
    fig.colorbar(im, fraction=0.046, pad=0.04)
    plt.title(f"Prediction (acc={acc})")

    plt.show()

# Making a submission

In [ ]:
import time

TEST_SET_FOLDER = os.path.abspath("data/test_set_images/")
SUBMISSION_NAME = f"submission_{time.time()}.csv"

SUBMISSION_PATH = os.path.abspath("submissions/" + SUBMISSION_NAME)

make_submission(SUBMISSION_PATH, TEST_SET_FOLDER)